# Multivariate Fuzzy C-means method: Implementation

## Equations

### $J= \sum_{i=1}^{c} \sum_{k=1}^{n} \sum_{j=1}^{p} \left(u_{ijk} \right)^{m} d_{ijk}$ - Objective function to minimize.

### $d_{ijk} = \left(x_{jk} - y_{ij} \right)^{2}$ - euclidian distance squared.

### $ y_{ij} = \frac{\sum_{k=1}^{n} \left(u_{ijk} \right)^{m} x_{jk}} {\sum_{k=1}^{n} \left(u_{ijk}\right)^{m}}$ - prototype coordinate of a given cluster in feature j.

### $ u_{ijk} =  \left[\sum_{h=1}^{c}\sum_{l=1}^{p} \left(\frac{d_{ijk}}{d_{hlk}}\right)^{(1/(m-1))}  \right]^{-1} $ - membership degree of pattern k in cluster $C_{i}$ on the feature j.

### $\delta_{ik} = \sum_{j=1}^{p} u_{ijk}$ - represents an aggregation measure for all the p features.

## Constraints:

### - $u_{ijk} \in [0, 1]$ for all i, j and k;
### - $0 < \sum_{j=1}^{p} \sum_{k=1}^{n} u_{ijk} < n$ for all i and
### - $\sum_{i=1}^{c}\sum_{j=1}^{p}u_{ijk} = 1$ for all k.

In [8]:
import numpy as np
import pandas as pd
from sklearn.metrics import adjusted_rand_score
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [9]:
df = sns.load_dataset('iris')
df["species"].replace({"setosa": 0, "versicolor": 1, "virginica": 2}, inplace=True)
df.columns = ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm", "Class"]
labels = df["Class"].values
df.drop("Class", axis=1, inplace=True)
dados = df.to_numpy()

In [10]:
class MFCM():
    def __init__(self, c, X, m):
        self.c = c
        self.n = X.shape[0]
        self.p = X.shape[1]
        self.m = m
    
    def initialize_u(self):
        return np.random.dirichlet(alpha=np.ones(self.c * self.p),
                                   size=self.n).reshape(self.n, self.c, self.p)
    
    def find_centroides(self, X, U):
        return np.sum((U ** self.m) * X[:, np.newaxis, :], axis=0) / np.sum(U ** self.m, axis=0)
    
    def get_distances(self, X, V): # as vezes tem umas distâncias muito pequenas
        return (X[:, np.newaxis, :] - V[np.newaxis, :, :]) ** 2
    
    def update_u(self, D):
        ratio = (D[:, np.newaxis, np.newaxis, :, :] / D[:, :, :, np.newaxis, np.newaxis]) ** (1/(self.m-1))
        return 1 / (np.sum(ratio, axis=(3, 4)))
    
    def get_objective_function(self, U, D):
        return np.sum((U ** self.m)*D)

In [11]:
def mfcm_run(dados, num_clusters, m=2, max_iter=10**3, epsilon=1e-5):
    mfcm = MFCM(c=num_clusters, X=dados, m=m) # create the MFCM object
    
    U = mfcm.initialize_u() # initialize the membership matrix
    
    for _ in range(max_iter):
        #print(_)
        centroids = mfcm.find_centroides(dados, U)
        D = mfcm.get_distances(dados, centroids)
        new_U = mfcm.update_u(D)
        if np.linalg.norm(U - new_U) < epsilon:
            break
        U = new_U
    
    #print(f"Membership matrix:\n{U}")
    Delta = np.sum(U, axis=2)  # summing over the second axis (variables `j`)
    #print(f"Delta matrix:\n{Delta}")
    
    return centroids, U, Delta

In [12]:
scaler = MinMaxScaler()

dados = scaler.fit_transform(dados)

dados

array([[0.22222222, 0.625     , 0.06779661, 0.04166667],
       [0.16666667, 0.41666667, 0.06779661, 0.04166667],
       [0.11111111, 0.5       , 0.05084746, 0.04166667],
       [0.08333333, 0.45833333, 0.08474576, 0.04166667],
       [0.19444444, 0.66666667, 0.06779661, 0.04166667],
       [0.30555556, 0.79166667, 0.11864407, 0.125     ],
       [0.08333333, 0.58333333, 0.06779661, 0.08333333],
       [0.19444444, 0.58333333, 0.08474576, 0.04166667],
       [0.02777778, 0.375     , 0.06779661, 0.04166667],
       [0.16666667, 0.45833333, 0.08474576, 0.        ],
       [0.30555556, 0.70833333, 0.08474576, 0.04166667],
       [0.13888889, 0.58333333, 0.10169492, 0.04166667],
       [0.13888889, 0.41666667, 0.06779661, 0.        ],
       [0.        , 0.41666667, 0.01694915, 0.        ],
       [0.41666667, 0.83333333, 0.03389831, 0.04166667],
       [0.38888889, 1.        , 0.08474576, 0.125     ],
       [0.30555556, 0.79166667, 0.05084746, 0.125     ],
       [0.22222222, 0.625     ,

In [13]:
def crisp_to_fuzzy(y, n_clusters): # transforma o dataset em fuzzy
    fuzzy_labels = np.zeros((len(y), n_clusters)) # cria uma array do dataset preenchida só com zeros
    for i, label in enumerate(y):
        fuzzy_labels[i, label] = 1 # com base na classe, o zero é substituído por um
    return fuzzy_labels

labels = crisp_to_fuzzy(labels, 3)
labels

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

In [14]:
def pertinence_distance(delta_k, delta_k_linha, c):
    # calcula a distância entre δ_k e δ_k' (matrizes de pertinência)
    return (1/c) * np.sum((delta_k - delta_k_linha) ** 2)

def fuzzy_rand_index(particao1, particao2, c):
    n = particao1.shape[0]
    total_sum = 0

    for k in range(n):
        for k_linha in range(k+1, n):
            if k != k_linha:
                # calcula a métrica para P
                delta_k = particao1[k]
                delta_k_prime = particao1[k_linha]
                EP = 1 - pertinence_distance(delta_k, delta_k_prime, c)

                # calcula a métrica para Q
                delta_k_Q = particao2[k]
                delta_k_prime_Q = particao2[k_linha]
                EQ = 1 - pertinence_distance(delta_k_Q, delta_k_prime_Q, c)

                total_sum += np.abs(EP - EQ) # soma a diferença absoluta entre EP e EQ

    denominador = n * (n - 1) / 2
    if denominador == 0:
        raise ValueError

    return 1- (total_sum / denominador)

In [21]:
def bootstrap_fr(dados, labels, num_clusters, m=2, n_bootstrap=1000, n_initializations=200, max_iter=10**3, epsilon=1e-5):
    n_samples = dados.shape[0]
    fr_indices = []

    for i in range(n_bootstrap):
        # Step 1: Generate bootstrap sample
        indices = np.random.choice(n_samples, n_samples, replace=True)
        bootstrap_sample = dados[indices]
        bootstrap_labels = labels[indices]

        best_U = None
        best_Delta = None
        best_objective = float('inf')

        # Step 2: Run the clustering algorithm with multiple initializations
        for _ in range(n_initializations):
            centroids, U, Delta = mfcm_run(bootstrap_sample, num_clusters, m=m, max_iter=max_iter, epsilon=epsilon)
            D = MFCM(num_clusters, bootstrap_sample, m).get_distances(bootstrap_sample, centroids)
            objective = MFCM(num_clusters, bootstrap_sample, m).get_objective_function(U, D)

            # Select the best result based on the objective function
            if objective < best_objective:
                best_U = U
                best_Delta = Delta
                best_objective = objective

        # Step 3: Calculate the Fuzzy Rand Index using the best result
        fr_index = fuzzy_rand_index(best_Delta, bootstrap_labels, num_clusters)
        print(f"Bootstrap iteration {i+1}/{n_bootstrap}: Fuzzy Rand Index = {fr_index}")
        fr_indices.append(fr_index)

    # Step 4: Calculate mean and standard deviation of Fuzzy Rand Index
    mean_fr = np.mean(fr_indices)
    std_fr = np.std(fr_indices)
    return mean_fr, std_fr

In [22]:
num_clusters = 3
mean_rand_index, std_rand_index, U_exemplo = bootstrap_fr(dados, labels, num_clusters)

print(f"Mean ARI: {mean_rand_index}")
print(f"Std ARI: {std_rand_index}")

Bootstrap iteration 1/1000: Fuzzy Rand Index = 0.5733248644086597
Bootstrap iteration 2/1000: Fuzzy Rand Index = 0.5609766871804995
Bootstrap iteration 3/1000: Fuzzy Rand Index = 0.5763015636362141
Bootstrap iteration 4/1000: Fuzzy Rand Index = 0.5613245712613497
Bootstrap iteration 5/1000: Fuzzy Rand Index = 0.5709844052934478
Bootstrap iteration 6/1000: Fuzzy Rand Index = 0.5705742282042512
Bootstrap iteration 7/1000: Fuzzy Rand Index = 0.5728285062980913
Bootstrap iteration 8/1000: Fuzzy Rand Index = 0.5747714568736054
Bootstrap iteration 9/1000: Fuzzy Rand Index = 0.5734505514167585
Bootstrap iteration 10/1000: Fuzzy Rand Index = 0.5736320920035534


KeyboardInterrupt: 